# Graphs, functions and tracing 

In [1]:
import time
import tensorflow as tf

In [12]:
def inefficient_square(x):
    for i in tf.range(100):
        i ** 2
    return x ** 2

Simplifies, optimizes, paralellizes and prune the function.

In [13]:
tf.function(inefficient_square)

In [14]:
tf_inefficient_square = tf.function(inefficient_square)

In [15]:
tf_inefficient_square.python_function(2)

4

Each tf function generates a new graph for each signature it receives. (signature = shape + data type). We have to be careful with this. This is true for tensor input. If we pass python numbers, it will generate a new graph for each call.

In [16]:
%timeit -n10 -r10 inefficient_square(2)

77.9 ms ± 1.98 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [17]:
%timeit -n10 -r10 tf_inefficient_square(2)

The slowest run took 53.09 times longer than the fastest. This could mean that an intermediate result is being cached.
3.18 ms ± 7.77 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [18]:
two = tf.constant(2)

In [19]:
%timeit -n10 -r10 tf_inefficient_square(two)

The slowest run took 28.96 times longer than the fastest. This could mean that an intermediate result is being cached.
2.04 ms ± 4.24 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [20]:
print(tf.autograph.to_code(inefficient_square))

def tf__inefficient_square(x):
    with ag__.FunctionScope('inefficient_square', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()

        def get_state():
            return ()

        def set_state(block_vars):
            pass

        def loop_body(itr):
            i = itr
            (ag__.ld(i) ** 2)
        i = ag__.Undefined('i')
        ag__.for_stmt(ag__.converted_call(ag__.ld(tf).range, (100,), None, fscope), None, loop_body, get_state, set_state, (), {'iterate_names': 'i'})
        try:
            do_return = True
            retval_ = (ag__.ld(x) ** 2)
        except:
            do_return = False
            raise
        return fscope.ret(retval_, do_return)



In [21]:
@tf.function
def sum_squares(n):
    s = 0
    for i in tf.range(n + 1):
        s += i ** 2
    return s

In [22]:
print(tf.autograph.to_code(sum_squares.python_function))

def tf__sum_squares(n):
    with ag__.FunctionScope('sum_squares', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()
        s = 0

        def get_state():
            return (s,)

        def set_state(vars_):
            nonlocal s
            (s,) = vars_

        def loop_body(itr):
            nonlocal s
            i = itr
            s = ag__.ld(s)
            s += (i ** 2)
        i = ag__.Undefined('i')
        ag__.for_stmt(ag__.converted_call(ag__.ld(tf).range, ((ag__.ld(n) + 1),), None, fscope), None, loop_body, get_state, set_state, ('s',), {'iterate_names': 'i'})
        try:
            do_return = True
            retval_ = ag__.ld(s)
        except:
            do_return = False
            raise
        return fscope.ret(retval_, do_return)



- use all tf tool possible, including tf.range or tf.sort
- otherwise, they will only be evaluated at tracing time
- for example. if we use np.random to compute a randon number, it will be computed only when tracing the new graph
- the same if our function has side effects, like logging or printing
- it will only capture for loops that iterate through tensorlfow things
- use vectorized operations! don't use loops!

In [23]:
import numpy as np

In [24]:
@tf.function
def random_thing(input):
    return np.random.randint(0,10)

For each number, a new graph.

In [30]:
random_thing(1), random_thing(1), random_thing(2), random_thing(3)

(<tf.Tensor: shape=(), dtype=int32, numpy=3>,
 <tf.Tensor: shape=(), dtype=int32, numpy=3>,
 <tf.Tensor: shape=(), dtype=int32, numpy=5>,
 <tf.Tensor: shape=(), dtype=int32, numpy=9>)

For each new signature, a new graph

In [32]:
random_thing(tf.constant([1])), random_thing(tf.constant([2])), random_thing(tf.constant([3]))

(<tf.Tensor: shape=(), dtype=int32, numpy=5>,
 <tf.Tensor: shape=(), dtype=int32, numpy=5>,
 <tf.Tensor: shape=(), dtype=int32, numpy=5>)

In [33]:
random_thing(tf.constant([1.0, 2.0]))

<tf.Tensor: shape=(), dtype=int32, numpy=6>

In [34]:
@tf.function
def printer(input):
    print('Hi :). This means that the graph has been retraced!')

In [35]:
printer(1)

Hi :). This means that the graph has been retraced!


In [36]:
printer(1)

In [37]:
printer(2)

Hi :). This means that the graph has been retraced!


In [38]:
printer(tf.constant([1.0, 2.0]))

Hi :). This means that the graph has been retraced!


In [39]:
printer(tf.constant([3.0, 2.0]))

In [40]:
printer(tf.constant([1.0, 2.0, 3.0, 4.0]))

Hi :). This means that the graph has been retraced!


loops are only traced once

In [48]:
import time

@tf.function
def sq(x):
    for i in tf.range(10):
        i ** 2
        print(':)')
        time.sleep(2)
        print(':D')
    return x ** 2

In [49]:
sq(1)

:)
:D


<tf.Tensor: shape=(), dtype=int32, numpy=1>

In [50]:
sq(1)

<tf.Tensor: shape=(), dtype=int32, numpy=1>